In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from onnx import *
import os
import scipy.io as sio

In [24]:
mat_contents=sio.loadmat('ACASXU_run2a_1_3_batch_2000.mat')
mat_contents2=sio.loadmat('neural_network_information_0.mat')
del mat_contents['__header__']
del mat_contents['__version__']
del mat_contents['__globals__']
del mat_contents2['__header__']
del mat_contents2['__version__']
del mat_contents2['__globals__']
mat_contents2

{'W': array([[array([[-2.83216e-01, -2.19524e-01],
        [-7.81048e-03, -2.15841e-01],
        [-3.04988e-01,  1.09021e+00],
        [-1.65135e+00,  1.14078e-01],
        [ 7.61641e-01, -6.05356e-01],
        [-3.87565e-01,  9.74517e-02],
        [-2.97733e-01, -4.46726e-01],
        [ 5.55820e-01, -3.11427e-01],
        [-5.99734e-01,  4.49554e-01],
        [-1.11791e+00, -5.87783e-02],
        [ 1.17564e+00, -1.70625e-01],
        [-7.99437e-02,  2.45560e+00],
        [-2.73269e-01, -9.18629e-02],
        [-4.38795e-03, -1.67910e-01],
        [-8.40506e-02,  2.46567e+00],
        [-2.04284e+00,  1.05592e-01],
        [-3.55080e-01, -1.33880e-01],
        [-1.39274e-01,  1.39553e+00],
        [-1.50978e-01, -8.29487e-02],
        [ 2.28684e+00, -1.08054e-02],
        [-4.61647e-01,  1.95655e+00],
        [-5.03975e-03, -3.58779e-01],
        [ 2.52663e+00, -4.46626e-01],
        [-2.02698e-01,  1.21761e-01],
        [ 2.55996e-03, -3.78915e-01],
        [ 2.06058e+00, -3.67221e-01],

In [25]:
W=mat_contents['W'].reshape((-1,1))
layerSizes=mat_contents['layer_sizes']
layerSizes1=mat_contents2['layer_sizes']
b=mat_contents['b'].reshape((-1,1))
W1=mat_contents2['W'].reshape((-1,1))
b1=mat_contents2['b'].reshape((-1,1))

In [26]:
def createWeightBiasLabels(W):
    weightNames=[]
    biasNames=[]
    weightPrefix="W"
    biasPrefix="B"
    for i in range(0,len(W)):
        weightNames.append(weightPrefix+str((i+1)))
        biasNames.append(biasPrefix+str((i+1)))
    return weightNames, biasNames

In [ ]:

num_iter=2*num_layers-1
nodeList.append(helper.make_node("FC", ["X", "W1", "B1"], ["H1"]))

In [ ]:
print(weightLabels)
biasLabels

In [ ]:
nodeList[0]

In [34]:
def createReluplexOnnx(W,b,layerSizes):
    num_layers=len(W)
    layer_name="FC"
    input_name="X"
    inputSize=layerSizes[0][0]
    outputSize=layerSizes[0][len(W)]
    output_name="Y"
    reluOutput="R"
    output1="O"
    reluOperation="ReLU"
    weightLabels, biasLabels=createWeightBiasLabels(W)
    nodeList=[]
    tensorList=[]
    if(int(inputSize)==1):
         tensorList.append(helper.make_tensor_value_info(input_name, TensorProto.FLOAT, [1]))
    else:
        tensorList.append(helper.make_tensor_value_info(input_name, TensorProto.FLOAT, [int(inputSize),1]))
    if(outputSize==1):
        outputList=[helper.make_tensor_value_info(output_name, TensorProto.FLOAT, [int(outputSize),1])]
    else:
        outputList=[helper.make_tensor_value_info(output_name, TensorProto.FLOAT, [1])]
    for i in range(0,num_layers):
        nodeList.append(helper.make_node(layer_name, [input_name, weightLabels[i], biasLabels[i]], [output1+str(i+1)]))
        input_name=output1+str(i+1)
        if(i==num_layers-1):
            continue
        else:
            nodeList.append(helper.make_node(reluOperation, [input_name], [reluOutput+str(i+1)]))
        input_name=reluOutput+str(i+1)
    for i in range(0,num_layers):
        weightMatrix=W[i][0]
        biasMatrix=b[i][0]
        if(weightMatrix.shape[0]==1 and weightMatrix.shape[1]==1):
            tensorList.append(helper.make_tensor_value_info(weightLabels[i], TensorProto.FLOAT, [1]))
        else:
            tensorList.append(helper.make_tensor_value_info(weightLabels[i], TensorProto.FLOAT, [weightMatrix.shape[0],weightMatrix.shape[1]]))
        if(biasMatrix.shape[0]==1 and biasMatrix.shape[1]==1):
            tensorList.append(helper.make_tensor_value_info(biasLabels[i], TensorProto.FLOAT, [1]))
        else:
            tensorList.append(helper.make_tensor_value_info(biasLabels[i], TensorProto.FLOAT, [biasMatrix.shape[0],biasMatrix.shape[1]]))
    print(nodeList)
    print(tensorList)
    print(outputList)
    graph=helper.make_graph(nodeList,"MLP",tensorList,outputList)
    model_def = helper.make_model(graph, producer_name='Sherlock->Onnx')
    return model_def
    
    

In [35]:
model=createReluplexOnnx(W,b,layerSizes)

[input: "X"
input: "W1"
input: "B1"
output: "O1"
op_type: "FC"
, input: "O1"
output: "R1"
op_type: "ReLU"
, input: "R1"
input: "W2"
input: "B2"
output: "O2"
op_type: "FC"
, input: "O2"
output: "R2"
op_type: "ReLU"
, input: "R2"
input: "W3"
input: "B3"
output: "O3"
op_type: "FC"
, input: "O3"
output: "R3"
op_type: "ReLU"
, input: "R3"
input: "W4"
input: "B4"
output: "O4"
op_type: "FC"
, input: "O4"
output: "R4"
op_type: "ReLU"
, input: "R4"
input: "W5"
input: "B5"
output: "O5"
op_type: "FC"
, input: "O5"
output: "R5"
op_type: "ReLU"
, input: "R5"
input: "W6"
input: "B6"
output: "O6"
op_type: "FC"
, input: "O6"
output: "R6"
op_type: "ReLU"
, input: "R6"
input: "W7"
input: "B7"
output: "O7"
op_type: "FC"
]
[name: "X"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 5
      }
      dim {
        dim_value: 1
      }
    }
  }
}
, name: "W1"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 50
      }
      dim {
   

In [15]:
new_model_path = os.path.join('/Users/Musau/Documents/Research/NeuralNetworkToolParsers/examples', 'reluExample.onnx')

In [16]:
new_model_path

'/Users/Musau/Documents/Research/NeuralNetworkToolParsers/examples/reluExample.onnx'

In [17]:
onnx.save(model, new_model_path)

In [44]:
def createSherlockOnnx(W,b,layerSizes):
    num_layers=len(W)
    layer_name="FC"
    input_name="X"
    inputSize=layerSizes[0][0]
    outputSize=layerSizes[0][len(W)]
    output_name="Y"
    reluOutput="R"
    output1="O"
    reluOperation="ReLU"
    weightLabels, biasLabels=createWeightBiasLabels(W)
    nodeList=[]
    tensorList=[]
    if(int(inputSize)==1):
         tensorList.append(helper.make_tensor_value_info(input_name, TensorProto.FLOAT, [1]))
    else:
        tensorList.append(helper.make_tensor_value_info(input_name, TensorProto.FLOAT, [int(inputSize),1]))
    if(outputSize==1):
        outputList=[helper.make_tensor_value_info(output_name, TensorProto.FLOAT, [1])]
    else:
        outputList=[helper.make_tensor_value_info(output_name, TensorProto.FLOAT, [int(outputSize),1])]
    for i in range(0,num_layers):
        nodeList.append(helper.make_node(layer_name, [input_name, weightLabels[i], biasLabels[i]], [output1+str(i+1)]))
        input_name=output1+str(i+1)
        if(i==num_layers-1):
            nodeList.append(helper.make_node(reluOperation, [input_name], [output_name]))
        else:
            nodeList.append(helper.make_node(reluOperation, [input_name], [reluOutput+str(i+1)]))
        input_name=reluOutput+str(i+1)
    for i in range(0,num_layers):
        weightMatrix=W[i][0]
        biasMatrix=b[i][0]
        if(weightMatrix.shape[0]==1 and weightMatrix.shape[1]==1):
            tensorList.append(helper.make_tensor_value_info(weightLabels[i], TensorProto.FLOAT, [1]))
        else:
            tensorList.append(helper.make_tensor_value_info(weightLabels[i], TensorProto.FLOAT, [weightMatrix.shape[0],weightMatrix.shape[1]]))
        if(biasMatrix.shape[0]==1 and biasMatrix.shape[1]==1):
            tensorList.append(helper.make_tensor_value_info(biasLabels[i], TensorProto.FLOAT, [1]))
        else:
            tensorList.append(helper.make_tensor_value_info(biasLabels[i], TensorProto.FLOAT, [biasMatrix.shape[0],biasMatrix.shape[1]]))
    graph=helper.make_graph(nodeList,"MLP",tensorList,outputList)
    model_def = helper.make_model(graph, producer_name='Sherlock->Onnx')
    return model_def

In [45]:
createSherlockOnnx(W1,b1,layerSizes1)

ir_version: 3
producer_name: "Sherlock->Onnx"
graph {
  node {
    input: "X"
    input: "W1"
    input: "B1"
    output: "O1"
    op_type: "FC"
  }
  node {
    input: "O1"
    output: "R1"
    op_type: "ReLU"
  }
  node {
    input: "R1"
    input: "W2"
    input: "B2"
    output: "O2"
    op_type: "FC"
  }
  node {
    input: "O2"
    output: "Y"
    op_type: "ReLU"
  }
  name: "MLP"
  input {
    name: "X"
    type {
      tensor_type {
        elem_type: FLOAT
        shape {
          dim {
            dim_value: 2
          }
          dim {
            dim_value: 1
          }
        }
      }
    }
  }
  input {
    name: "W1"
    type {
      tensor_type {
        elem_type: FLOAT
        shape {
          dim {
            dim_value: 100
          }
          dim {
            dim_value: 2
          }
        }
      }
    }
  }
  input {
    name: "B1"
    type {
      tensor_type {
        elem_type: FLOAT
        shape {
          dim {
            dim_value: 100
       

1